In [70]:
# !pip install -qU langchain-teddynote
# !pip install -qU "langchain[openai]"
# !pip install langchain-community
# !pip install -U langchain langchain-openai
# !pip install faiss-cpu
# !pip install -qU langchain-core langchain-upstage

In [71]:
import os
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from operator import itemgetter

In [72]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [73]:
upstage_api_key = os.getenv("UPSTAGE_API_KEY")

In [74]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("langchain_pjt_solar_hist")

LangSmith 추적을 시작합니다.
[프로젝트명]
langchain_pjt_solar_hist


In [75]:
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
)

In [76]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
# https://n.news.naver.com/mnews/article/123/0002356246?sid=101
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

In [77]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [78]:
# 벡터스토어를 생성합니다.
embeddings = UpstageEmbeddings(api_key=upstage_api_key, model="embedding-query")
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [79]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Previous Chat History:
{chat_history}

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [98]:
llm = ChatUpstage(model_name="solar-mini")


# 체인을 생성합니다.
rag_chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [249]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    # 현재 저장된 메시지 개수 출력
    print(f"저장된 메시지 개수 (변경 전): {len(store[session_ids].messages)}")

    # 메시지 리스트를 완전히 비우고 최근 10개만 유지
    tmp = store[session_ids].messages[-8:]
    store[session_ids].messages.clear()
    store[session_ids].messages.extend(tmp)

    # 변경 후 메시지 개수 출력
    print(f"저장된 메시지 개수 (변경 후): {len(store[session_ids].messages)}")

    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 대화를 기록하는 RAG 체인 생성
rag_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

In [250]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "부영그룹의 출산 장려 정책은?"},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 0
저장된 메시지 개수 (변경 후): 0


'부영그룹은 2021년 이후 태어난 직원 자녀에게 1억원씩 지원하고, 연년생과 쌍둥이 자녀가 있는 경우 총 2억원을 지원합니다. 또한, 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 출산 장려 정책을 내놓았습니다.'

In [251]:
# 1
#print(store["rag123"].messages)

In [252]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "앞서 대답한 내용을 영어로 번역해서 알려줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 2
저장된 메시지 개수 (변경 후): 2


'A company is offering a generous low birthrate policy by giving 100 million won to employees who give birth. This policy has been implemented since 2021 and will continue in the future. For multiple births, such as twins or consecutive births, an additional 100 million won will be provided. Furthermore, if an employee has a third child, they will be given a national housing unit. This initiative aims to support employees in raising their children and is expected to encourage more employees to have more children. The company has suggested that the government exempt these birth encouragement funds from taxes to further assist employees.'

In [253]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "육아휴직과 관련한 내용 알려줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 4
저장된 메시지 개수 (변경 후): 4


'주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.'

In [254]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "국민주택을 제공해주는 조건이 뭔지 알려줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 6
저장된 메시지 개수 (변경 후): 6


'셋째까지 낳는 경우 국민주택을 제공받을 수 있습니다.'

In [255]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "이 뉴스를 취재한 기자에 대해서 알려줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 8
저장된 메시지 개수 (변경 후): 8


'뉴스를 취재한 기자에 대한 정보는 제공되지 않습니다.'

In [256]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "내가 어떤걸 물어봤었는지 기억하면, 그 질문들을 순서대로 알려줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 10
저장된 메시지 개수 (변경 후): 8


'질문 리스트:\n1. 앞서 대답한 내용을 영어로 번역해서 알려줘.\n2. 육아휴직과 관련한 내용 알려줘.\n3. 국민주택을 제공해주는 조건이 뭔지 알려줘.\n4. 이 뉴스를 취재한 기자에 대해서 알려줘.'

In [257]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "2억원을 주는 조건이 뭔지 알려줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 10
저장된 메시지 개수 (변경 후): 8


'셋째까지 낳는 경우 국민주택을 제공받을 수 있습니다.'

In [258]:
rag_with_history.invoke(
    # 질문 입력
    {"question": "내가 어떤걸 물어봤었는지 기억하면, 그 질문들을 순서대로 알려줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "rag123"}},
)

[대화 세션ID]: rag123
저장된 메시지 개수 (변경 전): 10
저장된 메시지 개수 (변경 후): 8


'질문 리스트:\n1. 앞서 대답한 내용을 영어로 번역해서 알려줘.\n2. 육아휴직과 관련한 내용 알려줘.\n3. 국민주택을 제공해주는 조건이 뭔지 알려줘.\n4. 이 뉴스를 취재한 기자에 대해서 알려줘.\n5. 2억원을 주는 조건이 뭔지 알려줘.'

In [259]:
store["rag123"].messages[4]

HumanMessage(content='내가 어떤걸 물어봤었는지 기억하면, 그 질문들을 순서대로 알려줘.', additional_kwargs={}, response_metadata={})